In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import xml.etree.ElementTree as ET
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import l2


In [ ]:
batch_size = 32

input_shape = (224, 224, 3)


In [ ]:
dataset_path = '/content/drive/MyDrive/oocyte.v16i.voc'


In [ ]:
train_path = os.path.join(dataset_path, 'Train')
test_path = os.path.join(dataset_path, 'Test')
valid_path = os.path.join(dataset_path, 'Valid')

In [ ]:
def load_images_and_labels(image_dir, label_dir):
    images = []
    labels = []

    # List all XML files in the label directory
    xml_files = [filename for filename in os.listdir(label_dir) if filename.endswith('.xml')]
    print("Found XML Files:", xml_files)

    # Iterate over image files
    for image_filename in os.listdir(image_dir):
        if image_filename.endswith('.jpg'):
            image_path = os.path.join(image_dir, image_filename)
            label_filename = os.path.splitext(image_filename)[0] + '.xml'

            label_path = os.path.join(label_dir, label_filename)
            print("Checking Path:", label_path)  # Print the label path for troubleshooting

            # Load image
            image = load_img(image_path, target_size=input_shape[:2])
            image = img_to_array(image)
            images.append(image)

            if os.path.exists(label_path):
                # Parse XML label
                tree = ET.parse(label_path)
                root = tree.getroot()
                label_element = root.find('object/name')
                if label_element is not None:
                    label = label_element.text.strip()
                    labels.append(label)
                else:
                    labels.append("unknown")
            else:
                labels.append("unknown")

    return np.array(images), np.array(labels)

In [ ]:
train_images, train_labels = load_images_and_labels(train_path, train_path)
test_images, test_labels = load_images_and_labels(test_path, test_path)
valid_images, valid_labels = load_images_and_labels(valid_path, valid_path)


In [ ]:
label_encoder = LabelEncoder()

train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)
valid_labels_encoded = label_encoder.transform(valid_labels)

In [ ]:
num_classes = len(label_encoder.classes_)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes)
test_labels_categorical = to_categorical(test_labels_encoded, num_classes)
valid_labels_categorical = to_categorical(valid_labels_encoded, num_classes)


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_images, train_labels_categorical, epochs=10, validation_data=(valid_images, valid_labels_categorical))


Epoch 1/10
12/12 [==============================] - 334s 25s/step - loss: 25.8360 - accuracy: 0.5666 - val_loss: 15374.9756 - val_accuracy: 0.7662
Epoch 2/10
12/12 [==============================] - 317s 26s/step - loss: 1.9956 - accuracy: 0.6034 - val_loss: 52575.3633 - val_accuracy: 0.7662
Epoch 3/10
12/12 [==============================] - 314s 26s/step - loss: 1.6824 - accuracy: 0.6572 - val_loss: 483.5875 - val_accuracy: 0.7662
Epoch 4/10
12/12 [==============================] - 307s 25s/step - loss: 0.8136 - accuracy: 0.6657 - val_loss: 10950.6855 - val_accuracy: 0.7662
Epoch 5/10
12/12 [==============================] - 313s 26s/step - loss: 0.6269 - accuracy: 0.6884 - val_loss: 2046.1161 - val_accuracy: 0.7662
Epoch 6/10
12/12 [==============================] - 311s 26s/step - loss: 0.6021 - accuracy: 0.6941 - val_loss: 470.1448 - val_accuracy: 0.7662
Epoch 7/10
12/12 [==============================] - 310s 26s/step - loss: 0.5820 - accuracy: 0.7082 - val_loss: 102.1258 - val_a

In [ ]:
train_acc = history.history['accuracy'][-1]
print("Training Accuracy:", train_acc)

val_acc = history.history['val_accuracy'][-1]
print("Validation Accuracy:", val_acc)

test_loss, test_acc = model.evaluate(test_images, test_labels_categorical)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Training Accuracy: 0.7223796248435974
Validation Accuracy: 0.7662337422370911
3/3 [==============================] - 15s 5s/step - loss: 29.5732 - accuracy: 0.7067
Test Loss: 29.573150634765625
Test Accuracy: 0.7066666483879089


In [ ]:
model.save('path/to/save/model.h5')


In [ ]:

import joblib

# Save
joblib.dump(label_encoder, 'path/to/save/label_encoder.pkl')

# Load
label_encoder = joblib.load('path/to/save/label_encoder.pkl')


In [ ]:
!pip install streamlit tensorflow joblib
!pip install streamlit --quiet
!npm install -g npm

In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image, ImageOps
import joblib
import keras

st.title("Embryo Predictor")
st.header("Large Serpent Classifier")
st.text("Upload an image for classification")

# Load the label encoder
label_encoder = joblib.load('/content/path/to/save/label_encoder.pkl')

# Upload model.h5 file
model_file = st.file_uploader("/content/path/to/save/model.h5", type=["h5"])
model = None

if model_file is not None:
    model = keras.models.load_model(model_file)

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("Classifying...")

    st.write("Prediction:")
    if model is not None:
       if model is not None:
    # Preprocess the image and perform classification
    image = ImageOps.fit(image, (224, 224))
    image_array = img_to_array(image)
    image_array = image_array / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    prediction = model.predict(image_array)
    predicted_class_index = np.argmax(prediction)
    predicted_class_label = "bad" if predicted_class_index == 0 else "good"

    st.write(f"Predicted Class Label: {predicted_class_label}")
else:
    st.write("Please upload a valid model file.")



Writing app.py


In [ ]:
!streamlit run '/content/app.py'






  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.227.76.213:8501

